### Introduction

This project is aimed at exploring ways to extract information from the internet and store it in database.It is focused on extracting the information of golden fur ape NFT and then storing it in a mongodb database.

### Variables extracted
- Ape NFT
    - Name
    - Attributes
    - Price

### Business Outcome:

Data scraped from the internet can be a valuable resource for statistical analysis and research purposes. By storing this data in a database, it becomes possible to visualize all relevant information at once, resulting in significant time savings and increased efficiency.

For example, data scraping of Golden Fur NFTs can provide insights into the characteristics of each digital asset, enabling a more informed purchasing decision. The structured format of the stored details facilitates data analysis and enables efficient extraction of meaningful insights.

Moreover, the stored data can be used to perform advanced analytics and statistical modeling techniques such as clustering, regression, and predictive modeling. These techniques can identify patterns and relationships within the data and enable accurate predictions and forecasts based on historical trends.

Overall, the availability of structured data in a database can be a powerful tool for organizations seeking to make data-driven decisions, as it enables efficient data analysis and the extraction of valuable insights that can be used to inform business strategy and decision-making.

### Tools and Technologies Used
- Selenium
- Beautiful soup
- Mongodb






Importing libraries


In [4]:
from bs4 import BeautifulSoup
import requests
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import json
import pymongo 
import re

### APE NFT 

Loading selenium driver to open Bored Apes website with solid gold fur apes


In [ ]:
driver = webdriver.Chrome(executable_path=' ')
#This path will need to be changed for execution

driver.implicitly_wait(10)

driver.get("https://opensea.io/collection/boredapeyachtclub?search[sortAscending]=false&search[stringTraits][0][name]=Fur&search[stringTraits][0][values][0]=Solid%20Gold")
driver.implicitly_wait(30)


Function to save the page to files


In [7]:
def saveString(html, filename):
    try:
        file = open(filename,"w",encoding='utf-8')
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))

Fetching the source code of each of the page and storing it on the disk


In [ ]:
for i in range(1,9):    
    ape = driver.find_element(By.XPATH,"//*/div/div/div/div[5]/div/div[7]/div[3]/div[2]/div/div["+str(i)+"]/article/a/div[1]/div/div/div/div/span/img")
    ape.click()
    time.sleep(10)    
    driver.implicitly_wait(10)
    a="bayc_["+str(i)+"].htm"
    headers = {'User-Agent': 'Mozilla/5.0'}
    soup = driver.page_source
    saveString(soup,a)
    driver.set_page_load_timeout(10)
    driver.back()
    driver.implicitly_wait(10)

driver.quit()

Another approach to save and load the URL in selenium


In [ ]:
for i in range(1,9):    
    ape = driver.find_element(By.XPATH,"//*/div/div/div/div[5]/div/div[7]/div[3]/div[2]/div/div["+str(i)+"]/article/a")
    driver.get(ape.get_attribute('href'))

    time.sleep(10)    
    driver.implicitly_wait(10)
    a="bayc_["+str(i)+"].htm"
    headers = {'User-Agent': 'Mozilla/5.0'}
    soup = driver.page_source
    saveString(soup,a)
    driver.set_page_load_timeout(10)
    driver.back()
    driver.implicitly_wait(10)

driver.quit()


Nested dictionary to access and save ape attributes


In [1391]:
dict={}
for counter in range(1,9):
    file_html=loadString("bayc_["+str(counter)+"].htm")
    doc1 = BeautifulSoup(file_html, 'lxml')
    
    dict[str(counter)]={}
    dict[str(counter)]["Name"]=doc1.findAll("h1",attrs={"class":"sc-29427738-0 hKCSVX item--title"})[0].text
    z=doc1.findAll("div",attrs={"class":"sc-d6dd8af3-0 hkmmpQ item--property"})

    i=0
    while i<len(z):
        dict[str(counter)][z[i].contents[0].text]=z[i].contents[1].text
        i+=1
print(dict)


{'1': {'Name': '#4873', 'Background': 'Orange', 'Clothes': 'Tie Dye', 'Earring': 'Gold Hoop', 'Eyes': 'Scumbag', 'Fur': 'Solid Gold', 'Hat': 'Army Hat', 'Mouth': 'Bored Pipe'}, '2': {'Name': '#8861', 'Background': 'Yellow', 'Clothes': 'Black T', 'Eyes': 'Sad', 'Fur': 'Solid Gold', 'Hat': 'Bayc Flipped Brim', 'Mouth': 'Phoneme Vuh'}, '3': {'Name': '#3105', 'Background': 'Yellow', 'Clothes': 'Bandolier', 'Eyes': 'Closed', 'Fur': 'Solid Gold', 'Hat': 'Laurel Wreath', 'Mouth': 'Bored'}, '4': {'Name': '#544', 'Background': 'Army Green', 'Clothes': 'Tuxedo Tee', 'Earring': 'Silver Stud', 'Eyes': 'Eyepatch', 'Fur': 'Solid Gold', 'Hat': 'Sushi Chef Headband', 'Mouth': 'Phoneme L'}, '5': {'Name': '#811', 'Background': 'Yellow', 'Clothes': 'Biker Vest', 'Eyes': 'Sleepy', 'Fur': 'Solid Gold', 'Hat': "Seaman's Hat", 'Mouth': 'Rage'}, '6': {'Name': '#4580', 'Background': 'Orange', 'Clothes': 'Biker Vest', 'Eyes': '3d', 'Fur': 'Solid Gold', 'Hat': "Seaman's Hat", 'Mouth': 'Dumbfounded'}, '7': {'Name

Creating a new mongodb database named 'bayc' and storing all ape information

In [ ]:
client=pymongo.MongoClient()
db=client.get_database("bayc")
print(client.list_database_names())
ape_collection=db.create_collection("bayc")

Inserting documents in mongodb collection


In [1396]:
ape_collection.insert_one(dict)